In [1]:
# Import Libraries
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import zipfile
import csv
from tensorflow import keras
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import pandas as pd

In [2]:
# Extract the dataset
with zipfile.ZipFile('/content/drive/MyDrive/Dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data/')

In [3]:
# Increase precision of presented data for better side-by-side comparison
pd.set_option("display.precision", 8)
data_root = ('/content/data/Dataset/')
tf.config.optimizer.set_jit(True)
IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = str(data_root)

In [4]:
# Intialise Validation Set
datagen_kwargs = dict(rescale=1./255, validation_split=0.10)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    subset="validation",
    shuffle=True,
    target_size=IMAGE_SHAPE
)

Found 10000 images belonging to 2 classes.


In [5]:
# Initialise Training Set
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    subset="training",
    shuffle=True,
    target_size=IMAGE_SHAPE)

Found 90000 images belonging to 2 classes.


In [6]:
# Intialise labels
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

{'fake': 0, 'real': 1}


In [7]:
# Build the model
IMAGE_SIZE = 224

model = tf.keras.Sequential([
  hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4",
                 output_shape=[1280],
                 trainable=False,
                 ),
  tf.keras.layers.Dropout(0.4),
  #tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

model.build([None,224,224,3])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
Total params: 2,257,984
Trainable params: 0
Non-trainable params: 2,257,984
_________________________________________________________________


In [29]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3) #1e-3, 3e-4

model.compile(
  optimizer=optimizer,
  loss='categorical_crossentropy',
  metrics=['accuracy']
  )

In [ ]:
# Fit the model
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)

history = model.fit_generator(
    train_generator,
    epochs=10,
    verbose=1,
    validation_data=valid_generator,
    steps_per_epoch = steps_per_epoch, )

In [ ]:
# Print the final loss and accuracy
results = model.evaluate(valid_generator)
print("Final loss: {:.2f}".format(results[0]))
print("Final accuracy: {:.2f}%".format(results[0] * 100))

In [ ]:
# Convert to Tflite model for classification on mobile
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tf_lite_converter.convert()
open("/content/drive/MyDrive/model.tflite","wb").write(tflite_model)